In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Cannot open camera.")
else:
    while True:
        ret, frame = cap.read()

        if not ret:
            print("Cannot capture frame from the camera.")
            break

        equalized_frame = cv2.equalizeHist(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

        jet_colormap_frame = cv2.applyColorMap(equalized_frame, cv2.COLORMAP_JET)
        jet_colormap_frame = cv2.cvtColor(jet_colormap_frame, cv2.COLOR_BGR2RGB)

        color_balanced_frame = cv2.xphoto.createSimpleWB().balanceWhite(jet_colormap_frame)

        def logarithmic_transform(x):
            return int(255 * np.log1p(x) / np.log(256))

        log_transform_lut = np.array([logarithmic_transform(x) for x in range(256)], dtype=np.uint8)
        log_transformed_frame = cv2.LUT(color_balanced_frame, log_transform_lut)

        gamma = 1.5
        power_law_frame = np.uint8(255 * np.power(log_transformed_frame / log_transformed_frame.max(), gamma))

        frame = cv2.resize(frame, (220, 140))
        equalized_frame = cv2.resize(equalized_frame, (220, 140))
        jet_colormap_frame = cv2.resize(jet_colormap_frame, (220, 140))
        color_balanced_frame = cv2.resize(color_balanced_frame, (220, 140))
        log_transformed_frame = cv2.resize(log_transformed_frame, (220, 140)).astype(np.uint8)
        power_law_frame = cv2.resize(power_law_frame, (220, 140))

        hist_equalization_frame = cv2.merge([equalized_frame] * 3)
        heatmap_frame = jet_colormap_frame
        color_balance_frame = color_balanced_frame
        log_transform_frame = cv2.cvtColor(log_transformed_frame, cv2.COLOR_BGR2RGB)
        power_law_adjusted_frame = power_law_frame

        stacked_frames = np.hstack([frame, hist_equalization_frame, heatmap_frame, color_balance_frame,
                                    log_transform_frame, power_law_adjusted_frame])

        cv2.imshow('Enhanced Frames', stacked_frames)

        if cv2.waitKey(1) & 0xFF == ord('x'):
            break

    cap.release()
    cv2.destroyAllWindows()
